In [ ]:
!pip install protobuf
!pip install torcheval

In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
from pathlib import Path

PATH = '../data/'
data_root = Path(PATH)

import sys
sys.path.insert(1, PATH)

from model import MultilabelModel
from dataset import AbstractsDataset
from trainer import Trainer

MODEL = 'microsoft/deberta-v3-base'
MAX_SEQ_LEN = 230
BATCH_SIZE = 64
N_CLASSES = 155

# Initialize data

In [ ]:
train_data = pd.read_csv(data_root / 'train/train_data.csv')
test_data = pd.read_csv(data_root / 'test/test_data.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
train_dataset = AbstractsDataset(train_data, tokenizer, MAX_SEQ_LEN)
test_dataset = AbstractsDataset(test_data, tokenizer, MAX_SEQ_LEN)

In [ ]:
train_params = {
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
}
test_params = {
    'batch_size': BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
}

train_dataloader = DataLoader(train_dataset, **train_params)
test_dataloader = DataLoader(test_dataset, **test_params)

# Loading & fitting pretrained model

In [ ]:
config = {
    "n_classes": N_CLASSES,
    "dropout_rate": 0.1
}

model = MultilabelModel(
    MODEL,
    config=config
)

trainer_config = {
    "lr": 1e-5,
    "n_epochs": 18,
    "batch_size": BATCH_SIZE,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "seed": 42,
}

t = Trainer(trainer_config)
t.fit(model, train_dataloader, test_dataloader)